# Worksheet 07

Name:  Taoyu Chen
UID: U82740711

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[4.144424357998641, 6.855436309046279, 3.8735302307845325, 5.770955816672096, 5.168554279049571, 4.625439672496097, 3.13647132937946, 3.074083316434325, 2.6589586525627107, 4.90401933691874]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [3]:
mean = 8
stdev = 1
c2 = np.random.normal( mean , stdev , 10 ).tolist()
print(c2)

[7.189745483105459, 7.536872853421574, 7.889057752882981, 7.411941830633892, 8.23280506056468, 8.17733599200925, 7.794180905641904, 9.292010447618445, 6.764626397195623, 8.523721486141207]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [4]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[4.90401933691874, 8.523721486141207, 2.6589586525627107, 6.764626397195623, 3.074083316434325, 3.13647132937946, 9.292010447618445, 4.625439672496097, 5.168554279049571, 7.794180905641904]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Number of mixture components: 2
Means of mixture components: 5 and 8
Standard deviations of mixture components: 1

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [5]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2))]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[4.90401933691874, 2.6589586525627107, 3.074083316434325, 3.13647132937946, 4.625439672496097, 5.168554279049571]
[8.523721486141207, 6.764626397195623, 9.292010447618445, 7.794180905641904]
P(C_1) = 0.6,  P(C_2) = 0.4
mean_1 = 3.9279210978068178,  mean_2 = 8.093634809149295
var_1 = 0.9906946675195328,  var_2 = 0.8692536799929309


/Users/mirackchan/Library/Python/3.9/lib/python/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [6]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( prob_c[0] * pdf_i[0]/prob_x )
    prob_c1_x.append( prob_c[1] * pdf_i[1]/prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  4.90401933691874
probability of observing that point if it came from cluster 0 =  0.24784179858658997
probability of observing that point if it came from cluster 1 =  0.0005470528146541604
point =  8.523721486141207
probability of observing that point if it came from cluster 0 =  8.549827268588535e-06
probability of observing that point if it came from cluster 1 =  0.4060737971802223
point =  2.6589586525627107
probability of observing that point if it came from cluster 0 =  0.1772988929481532
probability of observing that point if it came from cluster 1 =  1.4917663316914281e-09
point =  6.764626397195623
probability of observing that point if it came from cluster 0 =  0.006677741504533481
probability of observing that point if it came from cluster 1 =  0.14261633490993583
point =  3.074083316434325
probability of observing that point if it came from cluster 0 =  0.27776247332753273
probability of observing that point if it came from cluster 1 =  2.6356530739657027e-08
point 

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [7]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]

var = [ sum(x[0] * x[1] for x in zip(prob_c0_x, map(lambda y : (y-mean[0])**2,data)) )/ sum(prob_c0_x), sum(x[0] * x[1] for x in zip(prob_c1_x, map(lambda y : (y-mean[1])**2,data)) )/ sum(prob_c1_x) ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(C_1) = 0.6058784609632566,  P(C_2) = 0.39412153903674335
mean_1 = 3.957657308979374,  mean_2 = 8.110054758162839
var_1 = 1.0674527489057313,  var_2 = 0.8698411018715476


g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [18]:
from scipy.stats import norm

prob_c0_x = [] # P(C_0 | X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        # print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( prob_c[0] * pdf_i[0]/prob_x )
    prob_c1_x.append( prob_c[1] * pdf_i[1]/prob_x )

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  4.90401933691874
point =  8.523721486141207
point =  2.6589586525627107
point =  6.764626397195623
point =  3.074083316434325
point =  3.13647132937946
point =  9.292010447618445
point =  4.625439672496097
point =  5.168554279049571
point =  7.794180905641904
4.90401933691874
Probability of coming from C_1 = 0.9986746601680708
Probability of coming from C_2 = 0.0013253398319292688

8.523721486141207
Probability of coming from C_1 = 0.0001491719110571509
Probability of coming from C_2 = 0.9998508280889428

2.6589586525627107
Probability of coming from C_1 = 0.9999999950378123
Probability of coming from C_2 = 4.96218756325845e-09

6.764626397195623
Probability of coming from C_1 = 0.11548837658902146
Probability of coming from C_2 = 0.8845116234109786

3.074083316434325
Probability of coming from C_1 = 0.9999999407827921
Probability of coming from C_2 = 5.921720792324959e-08

3.13647132937946
Probability of coming from C_1 = 0.9999999146111747
Probability of coming from C_2 = 8.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [20]:
for p in probs:
    print(p[0])
    # print("Probability of coming from C_1 = " + str(p[1]))
    # print("Probability of coming from C_2 = " + str(p[2]))
    if p[1]>[p2]:
        print("point is in cluster c1")
    else:
        print("point is in cluster c2")
    print()

### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      ?       |
| A  C |      ?       |
| A  D |      ?       |
| A  E |      ?       |
| B  C |      ?       |
| B  D |      ?       |
| B  E |      ?       |
| C  D |      ?       |
| C  E |      ?       |
| D  E |      ?       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.